In [16]:
#setup imports and make ure the files we care about exist
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os.path

#custom modules
import feature_extraction
import unsw_nb15_dataset

from importlib import reload #allow us to reload custom modules any time we like with modifications
reload(feature_extraction)
reload(unsw_nb15_dataset)

fe.test_cases()

In [ ]:
#first, load the dataset
data_set = unsw_nb15_dataset.UNSW_NB15(['/home/jaywalker/MachineLearning/PacketGAN/UNSW-NB15_1_clean.csv'],
                                       sequence_length=1, 
                                       transform=feature_extraction.build_feature_sequence_tensor)

#pick out the attack examples
data_set.use_only_category('DoS'); #in this case just look at DoS attack samples

#convert attack examples into time windows for training
#DONE by the UNSW_NB15 class at load-time

#convert each window into appropriate feature tensor
#DONE by the transform function passed to the UNSW_NB15 class at load-time

#associate a "masking" vector with each window
#TODO need to make a python file for my masking functions

#train GAN to replace masked packets
#TODO make a python file to contain the network class definitions and training funciton

#observe results
#...
#profit!